In [2]:
import random

In [3]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [4]:
print(problem(5,42))

[[0], [1], [0], [4], [0], [1], [4], [4], [4], [1, 3], [0, 1], [2], [1], [0], [0, 2], [2, 4], [3], [3], [4], [2, 4], [0], [1], [0, 1], [3], [2, 3]]


In [5]:
import numpy as np

In [6]:
from gx_utils import *

In [7]:
import logging


def greedy(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    while goal != covered:
        x = all_lists.pop(0)
        if not set(x) < covered:
            solution.append(x)
            covered |= set(x)

    logging.info(
        f"Greedy solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")

In [8]:
logging.getLogger().setLevel(logging.INFO)
for N in [5, 10, 20, 100, 500, 1000]:
    greedy(N)

INFO:root:Greedy solution for N=5: w=5 (bloat=0%)
INFO:root:Greedy solution for N=10: w=13 (bloat=30%)
INFO:root:Greedy solution for N=20: w=46 (bloat=130%)
INFO:root:Greedy solution for N=100: w=332 (bloat=232%)
INFO:root:Greedy solution for N=500: w=2162 (bloat=332%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)


In [9]:
import logging
from random import seed, choice
from typing import Callable

logging.basicConfig(format="%(message)s", level=logging.INFO)

In [10]:
class State:
    def __init__(self, data: np.ndarray):
        self._data = data.copy()
        self._data.flags.writeable = False

    def __hash__(self):
        return hash(bytes(self._data))

    def __eq__(self, other):
        return bytes(self._data) == bytes(other._data)

    def __lt__(self, other):
        return bytes(self._data) < bytes(other._data)

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

In [34]:
#return a new_state if elem contains element different from state
def result(state, elem):
    elem_set = set(elem)
    state_set = set(state.copy_data.tolist())
    intersection = list(elem_set - state_set)
    if intersection == []: return False

    new_state = State(np.array([state, elem]))
    print(State(np.array(new_state)))
    return State(np.array(new_state))



In [12]:
def search(
    initial_state: State,
    goal_test: Callable,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
    input_state: State

):  
    frontier = PriorityQueue()
    parent_state.clear()
    state_cost.clear()

    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0

    while state is not None and not goal_test(state):
        for elem in input_state:
            #index = input_state.index(elem)
            new_state = result(state, np.array(elem))
            
            # check if new node contains element different from ones in the state
            if not new_state or new_state == state:
                continue
            else:
                cost = 1
                if new_state not in state_cost and new_state not in frontier:
                    parent_state[new_state] = state
                    state_cost[new_state] = state_cost[state] + cost
                    frontier.push(new_state, p=priority_function(new_state))
                    logging.debug(f"Added new node to frontier (cost={state_cost[new_state]})")
                elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                    old_cost = state_cost[new_state]
                    parent_state[new_state] = state
                    state_cost[new_state] = state_cost[state] + cost
                    logging.debug(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
        if frontier:
            state = frontier.pop()
        else:
            state = None

    path = list()
    s = state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]

    logging.info(f"Found a solution in {len(path):,} steps; visited {len(state_cost):,} states")
    return list(reversed(path))

In [13]:
input_state = sorted(problem(5, seed=42), key=lambda l: len(l))
initial_state = State(np.array(input_state[0]))
print(input_state)
print(initial_state)

[[0], [1], [0], [4], [0], [1], [4], [4], [4], [2], [1], [0], [3], [3], [4], [0], [1], [3], [1, 3], [0, 1], [0, 2], [2, 4], [2, 4], [0, 1], [2, 3]]
[0]


In [14]:
GOAL = State(np.array(range(5)))
logging.info(f"Goal:\n{GOAL}")

def goal_test(state):
    return state == GOAL

INFO:root:Goal:
[0 1 2 3 4]


# Breadth-first

In [15]:
parent_state = dict()
state_cost = dict()

final = search(
    initial_state=initial_state,
    goal_test=goal_test,
    parent_state=parent_state,
    state_cost=state_cost,
    priority_function=lambda s: len(state_cost),
    unit_cost=lambda a: 1,
)

TypeError: search() missing 1 required positional argument: 'input_state'

# A*

In [ ]:
parent_state = dict()
state_cost = dict()


def h(state):
    return np.sum((state._data != GOAL._data) & (state._data > 0))


final = search(
    initial_state=initial_state,
    goal_test=goal_test,
    parent_state=parent_state,
    state_cost=state_cost,
    priority_function=lambda s: state_cost[s] + h(s),
    unit_cost=lambda a: 1,
)

In [17]:
l = [1]
m = [1,2,3]

if l in m:
    print('suca')
print('ciao')

prova_state = set(State(np.array(m)).copy_data().tolist())
m = np.array([[3,4],[5]])
intersection = list(set(m) - set(prova_state))

[prova_state.add(i) for i in intersection]
print(prova_state)
print(intersection)


#########################
prova = [5,6,7]
res = result(m, prova)
print(res)

ciao


/var/folders/k_/4h0dvkqj46g0m9nchy6tny8h0000gn/T/ipykernel_8338/3208662567.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  m = np.array([[3,4],[5]])


TypeError: unhashable type: 'list'

In [61]:

l = [1]
m = [[1],[2,3]]
m_1 = State(np.array([1]))
m_23 = State(np.array([2,3]))
m = State(np.array([m_1, m_23]))
#m = State(np.array(m))
#########################
prova = State(np.array([5,6,7]))
new_state = State(np.array([m.copy_data().tolist(), prova.copy_data().tolist()]))
print(State(np.array(new_state)))
m = m.copy_data().tolist()

[list([array([1]), array([2, 3])]) list([5, 6, 7])]


/var/folders/k_/4h0dvkqj46g0m9nchy6tny8h0000gn/T/ipykernel_8338/3867376264.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  new_state = State(np.array([[i for i in m.copy_data().tolist()], prova.copy_data().tolist()]))
